In [ ]:
# Import dependencies
import os as os
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
from tqdm import tqdm
import wandb
from wandb.keras import WandbMetricsLogger

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras import regularizers

from util import config

# Setup

In [ ]:
# Initialize a new W&B run
wandb.init(
    config={
        "bs": 12
        },
)

# Build and train the model

In [ ]:
# Define model
model = keras.Sequential()

# First convolutional layer
model.add(layers.Conv2D(16, kernel_size=(7, 7), strides=(1, 1), padding='same', kernel_regularizer=regularizers.L2(0.01), input_shape=(np.shape(X_tr)[1], np.shape(X_tr)[2], 1), name='conv1'))
model.add(layers.BatchNormalization(synchronized=True, name='bn1'))
model.add(layers.Activation('relu', name='relu1'))

# Second convolutional layer
model.add(layers.Conv2D(16, kernel_size=(7, 7), strides=(1, 1), padding='same', kernel_regularizer=regularizers.L2(0.01), name='conv2'))
model.add(layers.BatchNormalization(synchronized=True, name='bn2'))
model.add(layers.Activation('relu', name='relu2'))
model.add(layers.MaxPooling2D(pool_size=(5, 5), name='pool2'))
model.add(layers.Dropout(0.3, name='drop2'))

# Third convolutional layer
model.add(layers.Conv2D(32, kernel_size=(7, 7), strides=(1, 1), padding='same', kernel_regularizer=regularizers.L2(0.01), name='conv3'))
model.add(layers.BatchNormalization(synchronized=True, name='bn3'))
model.add(layers.Activation('relu', name='relu3'))
model.add(layers.MaxPooling2D(pool_size=(4, 100), name='pool3'))
model.add(layers.Dropout(0.3, name='drop3'))

# Flatten layer
model.add(layers.Flatten(name='flatten1'))

# First dense layer
model.add(layers.Dense(100, kernel_regularizer=regularizers.L2(0.01), name='fc1'))
model.add(layers.Activation('relu', name='relu4'))
model.add(layers.Dropout(0.3, name='drop4'))

# Output layer
model.add(layers.Dense(len(config.labels), kernel_regularizer=regularizers.L2(0.01), name='main_out'))
model.add(layers.Activation('softmax', name='softmax1'))

# Define the optimizer
adamOwn = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)

# Compile the model
model.compile(optimizer=adamOwn, loss='SparseCategoricalCrossentropy', metrics=['accuracy'])

# Print the model summary
print(model.summary())

In [ ]:
# Train the model
model_log = model.fit(
    X_tr, 
    Y_tr, 
    validation_data=(X_val, Y_val),
    batch_size=16,
    epochs=200, 
    callbacks = [WandbMetricsLogger()]
)

In [ ]:
# Plot the model log
plt.figure()
plt.plot(model_log.history['loss'])
plt.plot(model_log.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')